## Set 5
## 3. Word2Vec \*\*Principles**

#### Preparation

In [1]:
# download the helper function
!wget -O P3CHelpers.py https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/P3CHelpers.py

--2022-02-16 21:54:51--  https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/P3CHelpers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4939 (4.8K) [text/plain]
Saving to: ‘P3CHelpers.py’

P3CHelpers.py       100%[===================>]   4.82K  --.-KB/s    in 0.01s   

2022-02-16 21:54:51 (493 KB/s) - ‘P3CHelpers.py’ saved [4939/4939]



In [2]:
# download the dataset
!wget -O dr_seuss.txt https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/data/dr_seuss.txt

--2022-02-16 21:54:54--  https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/data/dr_seuss.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8810 (8.6K) [text/plain]
Saving to: ‘dr_seuss.txt’

dr_seuss.txt        100%[===================>]   8.60K  --.-KB/s    in 0s      

2022-02-16 21:54:54 (44.8 MB/s) - ‘dr_seuss.txt’ saved [8810/8810]



In [3]:
import numpy as np
from P3CHelpers import *
import torch
import torch.nn as nn
import torch.optim as optim

#### Problem D: 
Fill in the generate_traindata and find_most_similar_pairs functions.

In [89]:
def get_word_repr(word_to_index, word):
    """
    Returns one-hot-encoded feature representation of the specified word given
    a dictionary mapping words to their one-hot-encoded index.

    Arguments:
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        word:          Word whose feature representation we wish to compute.

    Returns:
        feature_representation:     Feature representation of the passed-in word.
    """
    unique_words = word_to_index.keys()
    # Return a vector that's zero everywhere besides the index corresponding to <word>
    feature_representation = np.zeros(len(unique_words))
    feature_representation[word_to_index[word]] = 1
    return feature_representation    

def generate_traindata(word_list, word_to_index, window_size=4):
    """
    Generates training data for Skipgram model.

    Arguments:
        word_list:     Sequential list of words (strings).
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        window_size:   Size of Skipgram window. Defaults to 2 
                       (use the default value when running your code).

    Returns:
        (trainX, trainY):     A pair of matrices (trainX, trainY) containing training 
                              points (one-hot-encoded vectors) and their corresponding output_word
                              (also one-hot-encoded vectors)

    """
    ##############################################################
    # TODO: Implement this function, populating trainX and trainY
    ##############################################################
    trainX = []
    trainY = []
    for i in range(len(word_list)):
      for j in range(-window_size, window_size + 1):
        if j != 0 and i + j >= 0 and i + j < len(word_list):
          word = word_list[i]
          window_word = word_list[i + j]
          trainX.append(get_word_repr(word_to_index, word))
          trainY.append(get_word_repr(word_to_index, window_word))
    return np.array(trainX), np.array(trainY)

In [93]:
from torch.nn.modules.activation import Softmax
def find_most_similar_pairs(filename, num_latent_factors):
    """
    Find the most similar pairs from the word embeddings computed from
    a body of text
    
    Arguments:
        filename:           Text file to read and train embeddings from
        num_latent_factors: The number of latent factors / the size of the embedding
    """
    # Load in a list of words from the specified file; remove non-alphanumeric characters
    # and make all chars lowercase.
    sample_text = load_word_list(filename)

    # Create word dictionary
    word_to_index = generate_onehot_dict(sample_text)
    print("Textfile contains %s unique words"%len(word_to_index))
    # Create training data
    trainX, trainY = generate_traindata(sample_text, word_to_index)

    ##############################################################
    # TODO: 1) Create and train model in Pytorch.      
    ##############################################################
    # vocab_size = number of unique words in our text file. Will be useful 
    # when adding layers to your neural network
    vocab_size = len(word_to_index)

    model = nn.Sequential(
        nn.Linear(vocab_size, num_latent_factors),
        nn.Linear(num_latent_factors, vocab_size),
    )

    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
    loss_fn = nn.CrossEntropyLoss()
    model.train()
    batch_size = 50
    num_epochs = 10

    trainXY = torch.from_numpy(np.array([trainX, trainY])).float()
    train_loader = torch.utils.data.DataLoader(trainXY, 
                                        batch_size=batch_size, shuffle=True)
    
    for epoch in range(num_epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()

    model.eval()

    ##############################################################
    # TODO: 2) Extract weights for hidden layer
    ##############################################################
    # set weights variable below
    print(list(model.parameters())[0].shape)
    print(list(model.parameters())[2].shape)
    for p in model.parameters():
      p.requires_grad = False
    weights = list(model.parameters())[2]

    # Find and print most similar pairs
    similar_pairs = most_similar_pairs(weights, word_to_index)
    for pair in similar_pairs[:30]:
        print(pair)

### Problem E-H:
Run your model on drseuss.txt and answer questions from E through H.

In [95]:
find_most_similar_pairs('dr_seuss.txt', 10)

Textfile contains 308 unique words
torch.Size([10, 308])
torch.Size([308, 10])
Pair(star, kite), Similarity: 0.9312736
Pair(kite, star), Similarity: 0.9312736
Pair(at, swish), Similarity: 0.92859495
Pair(swish, at), Similarity: 0.92859495
Pair(be, fox), Similarity: 0.92831886
Pair(fox, be), Similarity: 0.92831886
Pair(took, yop), Similarity: 0.914207
Pair(yop, took), Similarity: 0.914207
Pair(about, open), Similarity: 0.90022033
Pair(open, about), Similarity: 0.90022033
Pair(cannot, home), Similarity: 0.8887963
Pair(home, cannot), Similarity: 0.8887963
Pair(put, yop), Similarity: 0.88486445
Pair(tell, yop), Similarity: 0.8841802
Pair(left, grows), Similarity: 0.8830075
Pair(grows, left), Similarity: 0.8830075
Pair(just, goat), Similarity: 0.8800384
Pair(goat, just), Similarity: 0.8800384
Pair(look, now), Similarity: 0.8790283
Pair(now, look), Similarity: 0.8790283
Pair(you, think), Similarity: 0.8780294
Pair(think, you), Similarity: 0.8780294
Pair(live, kite), Similarity: 0.8768652
Pai